In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
import numpy as np

# Cargar el dataset
df = pd.read_csv('C:/Users/pepeb/Documents/EDEM_IA_MASTER/EDEM_IA_PROJECT_2/dataset.csv')

# Identificar las columnas categóricas y aplicar one-hot encoding
categorical_columns = df.select_dtypes(include=['object']).columns
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Escalar las columnas numéricas con MinMaxScaler
numeric_columns = df.select_dtypes(include=[np.number]).columns
scaler = MinMaxScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# Separar las características y la variable objetivo
X = df.drop(columns=['credit_score'])
y = df['credit_score']

# Definir el modelo
def build_model(input_shape):
    model = Sequential()
    model.add(Input(shape=(input_shape,)))
    model.add(Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(1))  # Capa de salida para regresión
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model

# Entrenar el modelo
model = build_model(X.shape[1])
history = model.fit(X, y, epochs=150, batch_size=32, validation_split=0.3, verbose=0)

# Función para hacer predicciones con entrada manual
def predict_credit_score(model, scaler, input_data):
    input_df = pd.DataFrame([input_data])
    input_df = pd.get_dummies(input_df)
    input_df = input_df.reindex(columns=X.columns, fill_value=0)
    input_scaled = scaler.transform(input_df)
    prediction = model.predict(input_scaled)
    return prediction[0][0]

# Ejemplo de entrada manual con todas las variables
sample_data = {
    'person_age': 70,
    'person_income': 4000,
    'person_emp_exp': 15,
    'loan_amnt': 10000,
    'loan_int_rate': 6.0,
    'loan_percent_income': 1000/4000,
    'cb_person_cred_hist_length': 1,
    'person_gender_male': 1,
    'person_gender_female': 0,
    'person_education_Master': 1,
    'person_education_High School': 0,
    'person_education_Bachelor': 0,
    'person_education_Associate': 0,
    'person_education_Doctorate': 0,
    'person_home_ownership_OTHER': 1,
    'person_home_ownership_RENT': 0,
    'person_home_ownership_OWN': 0,
    'person_home_ownership_MORTGAGE': 0,
    'loan_intent_VENTURE': 1,
    'loan_intent_PERSONAL': 0,
    'loan_intent_EDUCATION': 0,
    'loan_intent_MEDICAL': 0,
    'loan_intent_HOMEIMPROVEMENT': 0,
    'loan_intent_DEBTCONSOLIDATION': 0,
    'previous_loan_defaults_on_file_Yes': 1,
    'previous_loan_defaults_on_file_No': 0
}

# Hacer una predicción con la entrada manual
predicted_credit_score = predict_credit_score(model, scaler, sample_data)
print(f'Predicted Credit Score: {predicted_credit_score}')

KeyboardInterrupt: 